In [1]:
arb_str = '''
- keeping the same epochs + parameters and optimizing for loss average across training and validation set. i.e. adding new features (like transitioning from naive bigram model to self attention head and seeing the improvement in loss)
- optimizing for time per epoch, i.e. making the training faster while trying to keep the validation/training loss around the same.
- also: defining SMALL functions with specific requirements, and testing using examples
- pattern: defining smallest reproducible example

Ｕｎｉｃｏｄｅ! 🅤🅝🅘🅒🅞🅓🅔‽ 🇺‌🇳‌🇮‌🇨‌🇴‌🇩‌🇪! 😄 The very name strikes fear and awe into the hearts of programmers worldwide. We all know we ought to “support Unicode” in our software 
The Unicode Codespace Let’s start with some general orientation. The basic elements of Unicode—its “characters”, although that term isn’t quite right—are called code points. Code points are identified by number, customarily written in hexadecimal with the prefix “U+”, such as U+0041 “A” latin capital letter a or U+03B8 “θ” greek small letter theta. Each code point also has a short name, and quite a few other properties, specified in the Unicode Character Database.The set of all possible code points is called the codespace. The Unicode codespace consists of 1,114,112 code points. 
''' 

In [15]:
# INCREDIBLE! HOMEMADE GARBAGE!! :))))

from collections import defaultdict

def byte_pair_enc(x, vocab_size = 276):
    enc_x = list(x.encode("utf-8"))
    vocab_list = sorted(list(set(enc_x))) # original
    # to test: need to encode the string, check if its shorter, then decode
    
    new_tokens = {}
    start_idx = 256
    num_merges = vocab_size - 256
    while True: 
        freqs = defaultdict(lambda: 0)
        for i in range(len(enc_x)-1):
            freqs[tuple(enc_x[i:i+2])] += 1
        # for i in range(zip(enc_x, enc_x[1:]))
        
        most_freq = max(freqs, key=freqs.get)
        if freqs[most_freq] <= 1 or num_merges < 0:
            break
        
        new_tokens[start_idx] = most_freq
        nenc_x = []
        i = len(enc_x)-1
        while i >= 0:
            if tuple(enc_x[i-1:i+1]) == most_freq:
                nenc_x.append(start_idx)
                i -= 1
            else: 
                nenc_x.append(enc_x[i])
            i -= 1
        enc_x = nenc_x[::-1]
        
        start_idx += 1
        num_merges -= 1

    def decoder(x): 
        while max(x) >= 256: # we can cache these using dp, but too lazy
            dec_x = [] 
            for i in x:
                if i >= 256:
                    dec_x.extend(list(new_tokens[i]))
                else:
                    dec_x.append(i)
            x = dec_x
        return bytes(dec_x).decode("utf-8", errors="replace")
        
    return enc_x, decoder

enc, dec = byte_pair_enc(arb_str)
assert arb_str == dec(enc)

In [3]:
gl_str = '''
step 1: load and downsample from fineweb
step 2: decide between bpe regex and architectures by using experiments (i.e. vocab size/merge count, regex matches, etc.)
step 3: optimize current BPE implementation. have a validation set and compare working vs non-working

goals: 
- use gelu, add layer norm at the end
- use kroplos vs capybara scaling laws???
- optimize the training runs

- goal: 200M parameter model, 4B token count
'''

In [1]:
from datasets import load_dataset
import os

ds = load_dataset("parquet", data_files="/scratch/st-sielmann-1/nlin06/fineweb/sample/10BT/*.parquet", split="train", cache_dir="/scratch/st-sielmann-1/nlin06/hf_datasets_cache")

/home/nlin06/.conda/envs/base.3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
ds

Dataset({
    features: ['text', 'id', 'dump', 'url', 'date', 'file_path', 'language', 'language_score', 'token_count'],
    num_rows: 14868862
})

In [12]:
ds['text']

# 10b tokens, 14m rows
# planning on training on MAXIMUM 5B tokens
# maybe do 750m tokens

# 0.5 * 0.15
# 14868862
# sample 0.075 indices from total text, train the bpe


0.075

In [44]:
'''
improvements:
need to store which merges take place
'''

import pickle

def byte_pair_enc(x, num_merges=100):
    enc_x = list(x.encode("utf-8"))

    decoder_dict = {i: [i] for i in range(256)}
    encoder_dict = {}
    start_idx = 256
    for new_idx in range(start_idx, start_idx + num_merges): 
        freqs = defaultdict(lambda: 0)
        for f,s in zip(enc_x, enc_x[1:]):
            freqs[(f, s)] += 1
        
        most_freq = max(freqs, key=freqs.get)
        if freqs[most_freq] <= 1:
            break
        
        decoder_dict[new_idx] = decoder_dict[most_freq[0]] + decoder_dict[most_freq[1]]
        encoder_dict[most_freq] = new_idx
        
        nenc_x = []
        i = 0
        while i < len(enc_x):
            if tuple(enc_x[i:i+2]) == most_freq:
                nenc_x.append(new_idx)
                i += 1
            else: 
                nenc_x.append(enc_x[i])
            i += 1
        enc_x = nenc_x

    return encoder_dict, decoder_dict


enc, dec = byte_pair_enc(arb_str)

with open('ckpt/enc.pkl', 'wb') as f:
    pickle.dump(enc, f)
with open('ckpt/dec.pkl', 'wb') as f:
    pickle.dump(dec, f)


enc = encoder(enc)
dec = decoder(dec)

assert arb_str == dec(enc(arb_str)), dec(enc)
print("Success!")


Success!


In [2]:


def encoder(encoder_dict):
    def wrapper(x):
        x = list(x.encode("utf-8"))
        enc_x = x
        for k, v in encoder_dict.items():
            nenc_x = []
            i = 0
            while i < len(enc_x):
                if tuple(enc_x[i:i+2]) == k:
                    nenc_x.append(v)
                    i += 1
                else: 
                    nenc_x.append(enc_x[i])
                i += 1
            enc_x = nenc_x
        return enc_x
    return wrapper
    
def decoder(decoder_dict): 
    def wrapper(x):
        dec_x = [] 
        for i in x:
            dec_x.extend(decoder_dict[i])
        dec_x = bytes(dec_x).decode("utf-8", errors="replace")
        return dec_x
    return wrapper


In [5]:
import pickle

with open('ckpt/enc.pkl', 'rb') as f:
    enc = pickle.load(f)
with open('ckpt/dec.pkl', 'rb') as f:
    dec = pickle.load(f)

enc = encoder(enc)
dec = decoder(dec)

assert arb_str == dec(enc(arb_str)), dec(enc)
print("Success!")




Success!
